In [26]:
from pathlib import Path
import json

In [27]:
# convnextv2_results = "/data/pufanyi/project/SC4000/output/eval/convnextv2-cassava-leaf-disease-classification_full_train_2024-10-28-16-59-31/"
# # pufanyi/SC4000_ConvNeXtV2_base_balanced_12500
# convnextv2_balanced_results = "/data/pufanyi/project/SC4000/output/eval/convnextv2-cassava-leaf-disease-classification_full_train_2024-10-28-17-04-19"
# mobilenetv3_results = "/data/pufanyi/project/SC4000/output/eval/mobilenetv3-cassava-leaf-disease-classification_full_train_2024-10-28-17-16-02"
# vit_results = "/data/pufanyi/project/SC4000/output/eval/vit-cassava-leaf-disease-classification_full_train_2024-10-28-17-12-27"

In [61]:
convnextv2_results = "/data/pufanyi/project/SC4000/output/eval/convnextv2-cassava-leaf-disease-classification_full_validation_2024-10-28-18-07-13"

convnextv2_balanced_results = "/data/pufanyi/project/SC4000/output/eval/convnextv2-cassava-leaf-disease-classification_full_validation_2024-10-28-18-07-40"

mobilenetv3_results = "/data/pufanyi/project/SC4000/output/eval/mobilenetv3-cassava-leaf-disease-classification_full_validation_2024-10-28-18-08-46"

vit_results = "/data/pufanyi/project/SC4000/output/eval/vit-cassava-leaf-disease-classification_full_validation_2024-10-28-18-08-17"

In [62]:
def get_results(results_dir):
    results_path = Path(results_dir) / "result.json"
    with results_path.open() as f:
        results = json.load(f)
    results_dict = {}
    for result in results["result"]:
        results_dict[result["image_id"]] = result["logs"]
    return results_dict

In [63]:
def get_answer(results_dir):
    results_path = Path(results_dir) / "result.json"
    with results_path.open() as f:
        results = json.load(f)
    results_dict = {}
    for result in results["result"]:
        results_dict[result["image_id"]] = result["answer"]
    return results_dict

In [64]:
answer = get_answer(convnextv2_results)

In [65]:
results = {
    "convnextv2": get_results(convnextv2_results),
    "convnextv2_balanced": get_results(convnextv2_balanced_results),
    "mobilenetv3": get_results(mobilenetv3_results),
    "vit": get_results(vit_results),
}

In [66]:
for key, l in results.items():
    print(key, len(l))

convnextv2 1070
convnextv2_balanced 1070
mobilenetv3 1070
vit 1070


In [67]:
data = []

for image_id in answer:
    data.append(
        {
            "image_id": image_id,
            "answer": answer[image_id],
            **{model: res[image_id] for model, res in results.items()},
        }
    )

In [68]:
import pandas as pd

df = pd.DataFrame(data)

In [69]:
df.head()

,image_id,answer,convnextv2,convnextv2_balanced,mobilenetv3,vit
0,3384875851.jpg,3,"[-3.6255342960357666, -1.5589333772659302, 3.1...","[-1.0212113857269287, -1.1479402780532837, 1.4...","[0.0009760967805050313, 0.011872063390910625, ...","[-2.5688140392303467, -1.91754150390625, 2.486..."
1,1165503053.jpg,3,"[-3.0364162921905518, -1.7804832458496094, -0....","[-0.2553289234638214, -0.6120381355285645, -0....","[5.545440217247233e-05, 0.0009158043540082872,...","[-2.50654935836792, -1.3025351762771606, -0.29..."
2,2848007232.jpg,3,"[-2.9825754165649414, -1.5856062173843384, 0.4...","[-1.124006986618042, -0.8381869196891785, 0.63...","[0.0010300446301698685, 0.1493929773569107, 0....","[-1.7591975927352905, -1.7288086414337158, -0...."
3,731606877.jpg,3,"[-3.4885094165802, -1.2497446537017822, 3.8262...","[-0.3883638381958008, -0.42377811670303345, 1....","[2.6075305868289433e-05, 0.0006420664140023291...","[-3.52433180809021, -0.913058876991272, 3.8201..."
4,2268050032.jpg,3,"[-3.9142332077026367, -2.7957892417907715, -0....","[-0.2042015641927719, -1.1442240476608276, -0....","[7.402305072901072e-06, 1.2771777619491331e-05...","[-2.5766618251800537, -1.6451654434204102, -1...."


In [70]:
len(df)

1070

In [71]:
import datasets

data = datasets.Dataset.from_pandas(df)

In [72]:
data.push_to_hub("pufanyi/SC4000_eval_results", "full", split="validation")

Uploading the dataset shards: 100%|██████████| 1/1 [00:01<00:00,  1.68s/it]


CommitInfo(commit_url='https://huggingface.co/datasets/pufanyi/SC4000_eval_results/commit/fdfbaba515c1729e4c5d6ff93f4dbd2a37f099c4', commit_message='Upload dataset', commit_description='', oid='fdfbaba515c1729e4c5d6ff93f4dbd2a37f099c4', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/pufanyi/SC4000_eval_results', endpoint='https://huggingface.co', repo_type='dataset', repo_id='pufanyi/SC4000_eval_results'), pr_revision=None, pr_num=None)

In [74]:
from sklearn.model_selection import train_test_split


train_data, validation_data = train_test_split(df, test_size=0.2, random_state=42)

In [75]:
train_data = datasets.Dataset.from_pandas(train_data)
validation_data = datasets.Dataset.from_pandas(validation_data)

In [76]:
train_data.push_to_hub("pufanyi/SC4000_eval_results", "val", split="train")
validation_data.push_to_hub("pufanyi/SC4000_eval_results", "val", split="validation")

Uploading the dataset shards: 100%|██████████| 1/1 [00:01<00:00,  1.14s/it]


CommitInfo(commit_url='https://huggingface.co/datasets/pufanyi/SC4000_eval_results/commit/d5e8b3ccf91bcdbf31737703124f78b2bdae78cd', commit_message='Upload dataset', commit_description='', oid='d5e8b3ccf91bcdbf31737703124f78b2bdae78cd', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/pufanyi/SC4000_eval_results', endpoint='https://huggingface.co', repo_type='dataset', repo_id='pufanyi/SC4000_eval_results'), pr_revision=None, pr_num=None)